In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/openai/CLIP.git > /dev/null #run only once / runtime

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-6ylohzre


In [ ]:
from zipfile import ZipFile as zf, BadZipFile

# Ensure the file path is correct and the file exists
file_path = "/content/drive/MyDrive/Samples.zip"

# Add error handling to provide more context
try:
    with zf(file_path, "r") as zip_ref:
        zip_ref.extractall("/")
except FileNotFoundError:
    print(f"Error: File not found at path: {file_path}")
except BadZipFile:
    print(f"Error: Invalid or corrupted zip file at path: {file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

df = pd.read_csv('/content/drive/MyDrive/TestData/data - Train.csv')

df = df.drop(columns=["Image"])

X = df.drop(columns=["Actual"])
y = df["Actual"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='rbf'))
])

svm_pipeline.fit(X_train, y_train)

# Save the trained model pipeline
joblib.dump(svm_pipeline, '/content/svm_model.pkl')

# Load the saved model pipeline
loaded_model = joblib.load('/content/svm_model.pkl')

# Use it to make predictions
y_pred = loaded_model.predict(X_test)
print(classification_report(y_test, y_pred))


                         precision    recall  f1-score   support

                 Benign       0.81      0.17      0.29        75
      [Malignant] Pre-B       0.82      0.80      0.81       138
      [Malignant] Pro-B       0.42      0.51      0.46        97
[Malignant] early Pre-B       0.46      0.59      0.52       144

               accuracy                           0.57       454
              macro avg       0.63      0.52      0.52       454
           weighted avg       0.62      0.57      0.56       454



In [ ]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("/content/drive/MyDrive/TestData/data - Train.csv")

label_encoder = LabelEncoder()
df["Actual"] = label_encoder.fit_transform(df["Actual"])
joblib.dump(label_encoder, "label_encoder.pkl")


# Optionally print to check
print(df["Actual"].value_counts())
print(label_encoder.classes_)  # Shows which class maps to which number


Actual
3    685
1    668
2    557
0    357
Name: count, dtype: int64
['Benign' '[Malignant] Pre-B' '[Malignant] Pro-B'
 '[Malignant] early Pre-B']


In [ ]:
#WITH VALIDATION

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from PIL import Image
from torchvision import transforms
import pandas as pd
import clip

# --- Load CSVs ---
train_df = pd.read_csv("/content/drive/MyDrive/TestData/data - Train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/TestData/data - Val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/TestData/data - Test.csv")

# --- Encode Labels ---
label_encoder = LabelEncoder()
train_df["Actual"] = label_encoder.fit_transform(train_df["Actual"])
val_df["Actual"] = label_encoder.transform(val_df["Actual"])
test_df["Actual"] = label_encoder.transform(test_df["Actual"])
joblib.dump(label_encoder, "label_encoder.pkl")

# --- Dataset Class ---
class BloodSmearDataset(Dataset):
    def __init__(self, dataframe, image_transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.transform = image_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        image = Image.open(row["Image"]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        tabular_features = torch.tensor([
            row["Count"], row["Mean"], row["Max"], row["Std"]
        ], dtype=torch.float)
        label = torch.tensor(row["Actual"], dtype=torch.long)
        return image, tabular_features, label

# --- Transforms ---
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

# --- Datasets & Loaders ---
train_dataset = BloodSmearDataset(train_df, image_transform)
val_dataset = BloodSmearDataset(val_df, image_transform)
test_dataset = BloodSmearDataset(test_df, image_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

# --- Load CLIP ---
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, _ = clip.load("ViT-B/32", device=device)
clip_model.eval()

# --- Model ---
class VLMClassifier(nn.Module):
    def __init__(self, image_feature_dim=512, tabular_dim=4, hidden_dim=128, num_classes=4):
        super().__init__()
        self.tabular_net = nn.Sequential(
            nn.Linear(tabular_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.classifier = nn.Sequential(
            nn.Linear(image_feature_dim + hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, image_features, tabular_features):
        tabular_out = self.tabular_net(tabular_features)
        combined = torch.cat([image_features, tabular_out], dim=1)
        return self.classifier(combined)

vlm_model = VLMClassifier(num_classes=len(label_encoder.classes_)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vlm_model.parameters(), lr=1e-4)

# --- Training with Validation ---
num_epochs = 30
best_val_loss = float("inf")

for epoch in range(num_epochs):
    vlm_model.train()
    total_loss = 0.0
    for images, tabular_data, labels in train_loader:
        with torch.no_grad():
            image_features = clip_model.encode_image(images.to(device)).float()
        outputs = vlm_model(image_features, tabular_data.to(device))
        loss = criterion(outputs, labels.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)

    # --- Validation ---
    vlm_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, tabular_data, labels in val_loader:
            image_features = clip_model.encode_image(images.to(device)).float()
            outputs = vlm_model(image_features, tabular_data.to(device))
            loss = criterion(outputs, labels.to(device))
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(vlm_model.state_dict(), "/content/best_vlm_model.pth")
        print(f"Epoch {epoch+1}: 🟢 Saved best model. Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")
    else:
        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")


100%|████████████████████████████████████████| 338M/338M [00:03<00:00, 108MiB/s]


Epoch 1: 🟢 Saved best model. Train Loss = 1.1526, Val Loss = 1.0073
Epoch 2: 🟢 Saved best model. Train Loss = 0.9149, Val Loss = 0.8087
Epoch 3: 🟢 Saved best model. Train Loss = 0.7404, Val Loss = 0.6590
Epoch 4: 🟢 Saved best model. Train Loss = 0.6081, Val Loss = 0.5475
Epoch 5: 🟢 Saved best model. Train Loss = 0.5053, Val Loss = 0.4623
Epoch 6: 🟢 Saved best model. Train Loss = 0.4340, Val Loss = 0.4041
Epoch 7: 🟢 Saved best model. Train Loss = 0.3776, Val Loss = 0.3556
Epoch 8: 🟢 Saved best model. Train Loss = 0.3370, Val Loss = 0.3176
Epoch 9: 🟢 Saved best model. Train Loss = 0.3010, Val Loss = 0.2852
Epoch 10: 🟢 Saved best model. Train Loss = 0.2720, Val Loss = 0.2675


In [ ]:
#WITHOUT VALIDATION

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from PIL import Image
from torchvision import transforms
import pandas as pd
import clip

df = pd.read_csv("/content/drive/MyDrive/TestData/data - Train.csv")
label_encoder = LabelEncoder()
df["Actual"] = label_encoder.fit_transform(df["Actual"])

joblib.dump(label_encoder, "label_encoder.pkl")

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["Actual"])

class BloodSmearDataset(Dataset):
    def __init__(self, dataframe, image_transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.transform = image_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        image = Image.open(row["Image"]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        tabular_features = torch.tensor([
            row["Count"], row["Mean"], row["Max"], row["Std"]
        ], dtype=torch.float)
        label = torch.tensor(row["Actual"], dtype=torch.long)
        return image, tabular_features, label

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

train_dataset = BloodSmearDataset(train_df, image_transform)
test_dataset = BloodSmearDataset(test_df, image_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, _ = clip.load("ViT-B/32", device=device)
clip_model.eval()

class VLMClassifier(nn.Module):
    def __init__(self, image_feature_dim=512, tabular_dim=4, hidden_dim=128, num_classes=4):
        super().__init__()
        self.tabular_net = nn.Sequential(
            nn.Linear(tabular_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.classifier = nn.Sequential(
            nn.Linear(image_feature_dim + hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, image_features, tabular_features):
        tabular_out = self.tabular_net(tabular_features)
        combined = torch.cat([image_features, tabular_out], dim=1)
        return self.classifier(combined)

vlm_model = VLMClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vlm_model.parameters(), lr=1e-4)

# Training loop with saving
num_epochs = 30
best_loss = float("inf")
for epoch in range(num_epochs):
    vlm_model.train()
    total_loss = 0.0
    for images, tabular_data, labels in train_loader:
        with torch.no_grad():
            image_features = clip_model.encode_image(images.to(device)).float()
        outputs = vlm_model(image_features, tabular_data.to(device))
        loss = criterion(outputs, labels.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(vlm_model.state_dict(), "/content/best_vlm_model.pth")
        print(f"Epoch {epoch+1}: 🟢 Saved best model. Loss = {avg_loss:.4f}")
    else:
        print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")


In [ ]:
from google.colab import files
files.download("/content/best_vlm_model.pth")


In [ ]:
# Load best model
vlm_model.load_state_dict(torch.load("/content/best_vlm_model.pth"))
vlm_model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, tabular_data, labels in test_loader:
        image_features = clip_model.encode_image(images.to(device)).float()
        outputs = vlm_model(image_features, tabular_data.to(device))
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convert to actual class names
all_pred_names = label_encoder.inverse_transform(all_preds)
all_label_names = label_encoder.inverse_transform(all_labels)

print(classification_report(all_label_names, all_pred_names))


                         precision    recall  f1-score   support

                 Benign       0.98      0.92      0.95        71
      [Malignant] Pre-B       0.99      0.99      0.99       134
      [Malignant] Pro-B       0.99      1.00      1.00       112
[Malignant] early Pre-B       0.96      0.99      0.97       137

               accuracy                           0.98       454
              macro avg       0.98      0.97      0.98       454
           weighted avg       0.98      0.98      0.98       454

